In [1]:
from IPython.display import HTML

js = """<script>IPython.notebook.kernel.execute("file_name=('" + IPython.notebook.notebook_name + "')");</script>"""
display(HTML(js))

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 19章 1つの名義変数で量的予測変数を予測する

## はじめに
- 本章で扱うのは、量的被予測変数と名義的予測変数のデータ構造
- 例
    - 政治的な党派関係　→ 収入を予測
    - 性行動のカテゴリ　→　寿命を予測（今回扱うこと）
- このような種類のデータ構造に関する伝統的な手法
    - 一要因分散分析（ANOVA）、または一元配置ANOVAと呼ばれている
    - これらの手法を用いて、予測変数によって、被予測変数の平均値に相違があるのかを調べる。
        - 政治的な党派ごとに収入が異なるか、など
- 今回紹介するベイジアンアプローチは、この伝統的な分散分析モデルを階層的に一般化したもの
- また、名義的予測変数だけでなく、量的予測変数(共変量)も伴う状況を考える。
    - このようなデータ構造については、伝統的な手法として共分散分析(ANCOVA)が用いられている。
    - こちらについても、伝統的なモデルの一般化を考える
- 1つの名義的予測変数の線形関数
- データのノイズを表すための、正規分布に従う恒等のリンク関数も含む

## 19.1. 量的データの複数の群の表記

<img src="./img/fig19_1.png" width="500">

- 各群のデータは「中心傾向周りの」「ランダムな変動」として記述されるものを扱う
- 各群の中心傾向は「全体のベースライン($\beta_0$)からの偏向」として捉える
- なお、ここではデータは、標準偏差の等しい正規分布に従っているとする(後ほど、標準偏差が群ごとに異なる場合を考える)

- 各群ごとの予測値 $\mu$ は、下記の式で表される。全体のベースラインに群の変更を加えている
$$
\begin{aligned} \mu & = \beta _ { 0 } + \sum _ { j } \beta _ { [ j ] } x _ { [ j ] } \\ & = \beta _ { 0 } + \vec { \beta } \cdot \vec { x } \end{aligned}
$$

- また、全体のベースラインからのカテゴリの偏向は、足しあげると0になるようになっている。（ゼロ和条件）

$$
\sum _ { j } \beta _ { [ j ] } = 0
$$

- ゼロ和条件がないと完全ではない。ゼロ和条件を満たすようにするのは簡単(式19.3)

- 図19.1で示した記述的モデルは、古典的なANOVAで用いた伝統的なもの。
- より一般的なモデルはベイズ用ソフトウェアで直接実装される。
    - 外れ値は正規分布の代わりに裾の重いノイズ分布(t分布など)を用いることによって取り込み可能
    - 異なる群に異なる標準偏差を与えることができる

## 19.2. 伝統的な分散分析
- 「分散分析」：データ全体の分散を群内分散と群間分散に分解するという意味
- 代数的にいうと、
    - 「全体平均値からの偏差の二乗和」＝「それぞれの群平均値からの偏差の二乗和」＋「全体平均値からの群平均の偏差の二乗和」
    - つまり、「全体の分散」＝「群内分散」＋「群間分散」
    - ANOVAはこれらの代数的な側面を表している。
- ベイズ的手法は、要素の分散を推定できはするが、このような代数的な関係は使わない。
- そのため、ベイジアンアプローチとANOVAは異なるものである。しかし、類似はしている。

### 伝統的なANOVAについて
- 帰無仮説：各群の平均は等しい（なお、ANOVAは3群以上の場合に用いる）
- p値に基づいて上記を決定する
- p値は、帰無仮説からの擬似サンプリングによって計算される
- なお、伝統的なANOVAでは、帰無仮説は下記を前提としている
    1. 各群のデータが正規分布に従っている
    2. 各群、同じ標準偏差であること（分散の等質性）
- サンプリングする統計量は、群内分散と群間分散の比。これをF比と呼ぶ。サンプリング分布はF分布と呼ばれている。
- なお、この2つの前提は慣例として定着しているため、本書の基本モデルでも前提として持ってくる。
- ただ、ベイズ用ソフトウェアではこれらの前提は緩和される。
    - 各群によって異なる分散を用いることができる
    - 正規分布以外も群内のデータを表すのに用いることができる

# 19.3. 階層ベイジアン・アプローチ
- 再掲
<img src="./img/fig19_1.png">

- 私たちの目標：ベイズ的な枠組みで、上述したパラメータを推定することである
- 式19.1.
$$
\begin{aligned} \mu & = \beta _ { 0 } + \sum _ { j } \beta _ { [ j ] } x _ { [ j ] } \\ & = \beta _ { 0 } + \vec { \beta } \cdot \vec { x } \end{aligned}
$$

- 全てのパラメータは図19.2のように、構造化された事前分布を与えられる必要がある。

### 階層的ダイアグラム
<img src="./img/fig19_2.png" width="500">

- 下から見ていく
- データ $y_i$が予測値 $\mu_i$を中心に、正規分布に従っている。その正規分布の標準偏差には$\sigma_y$には一様事前分布が与えられている
- ベースラインパラメータ $\beta_0$はデータにとって広い範囲の正規事前分布を与える
- 群の偏向パラメータ $\beta_j$には平均0の正規事前分布を与える
    - なぜなら偏向パラメータの総和は0になるため。

- ベイジアンアプローチでの目新しいところは $\sigma_{\beta}$ (偏向パラメータ分布の標準偏差)
- 事前分布を何か選ぶこともできるし、定数として設定することも可能。
    - $\sigma_{\beta}$を大きな定数として設定することで、伝統的なANOVAにもっとも近い結果となる
- 定数として $\sigma_\beta$を固定する代わりに、データから推定することもできる。

### 19.3.1. JAGSでの実装
- スルー

### 19.3.2. 例：セックスと死亡について
#### 実験概要
- ショウジョウバエでの実験
- オスについて性行為がどう負担になるかを調べる。
- 5つの群について、25匹ずつのオスのショウジョウバエが割り当てられた
    - ゼロ：相手がいないオスの群
    - 妊娠1：妊娠したメス1匹のみとつがいにされたオスの群
    - 妊娠8：妊娠したメス8匹とつがいにされた群
    - 処女1：未交尾のメス1匹のみとつがいにされた群
    - 処女8：未交尾のメス8匹とつがいにされた群
- 図19.3を見ると、性行為が増えるに従い、寿命が減るということを示唆している(図19.3 の赤丸)
- 目標：寿命と群間の差異の程度を推定すること。

<img src="./img/fig19_3_1.png" width="500">

#### グラフの概要
- 図19.2　の階層的ダイアグラムを元に、事後予測分布を求めたのが図19.3
- MCMCチェインのある点における「群の平均」と「標準偏差」を持つ正規曲線を繰り返しプロットした。
- この正規曲線を髭と呼んでいる
    - 髭がもじゃもじゃしている：正規曲線がばらけていて、線がたくさん見える状態
    - 髭がもじゃもじゃしていない：正規曲線がまとまっていて、線が少なく見える状態
- 髭のもじゃもじゃが推定の不確実性を表す。その振れ幅が標準偏差の値を表している。
- 横幅の長い、薄い髭：高い確実性で推定される大きな標準偏差
- 横幅の短い、もじゃもじゃした髭：低い確実性で推定される小さな標準偏差
    - つまり、髭のもじゃもじゃは、平均と標準偏差の不確実性によって影響を受ける

#### 結果の評価
- 事後予測分布にそこまで大きな外れ値はなく、各群内でのデータの散らばりは事後正規分布の幅に合致していることを明らかにしている
    - 分散の等質性を目視で確認する場合、データの散らばりがどう見えるかはサンプルサイズによるため注意が必要
- 正規分布の頂点で示された群平均の範囲を見ると、
    - 処女8群は他の群より低い
    - 処女1群も統制群よりも低そう
    - これらを確かめるために群平均の差を分析する

### 19.3.3. 対比
#### 私達が興味を持っていることと、それを知るにはどうすればよいか
- 研究者は、群間の比較か、群の集合と集合の比較に興味を持っている。
    - 4つの操作群とゼロ統制群の違い
    - 3つの統制群（ゼロ、妊娠1、妊娠8）の平均値と、2つの性行為が活発な群（処女1、処女8）
- MCMCチェインの各ステップから群の平均値の組をえることができる。
- もし群１と群２の平均の差に興味があれば、MCMCチェインの各ステップで計算すれば良い
$$
\begin{aligned} \mu _ { 1 } - \mu _ { 2 } & = \left( \beta _ { 0 } + \beta _ { 1 } \right) - \left( \beta _ { 0 } + \beta _ { 2 } \right) \\ & = ( + 1 ) \cdot \beta _ { 1 } + ( - 1 ) \cdot \beta _ { 2 } \end{aligned}
$$
- この計算式を見ると、ベースラインは計算から除外され、群間の偏向は重み付けされた群偏差の和となっていることがわかる。
- もし群１〜３の平均値と群４〜５の平均値の差に興味があれば下記。
$$
\begin{aligned} \left( \mu _ { 1 } + \right. & \mu _ { 2 } + \mu _ { 3 } ) / 3 - \left( \mu _ { 4 } + \mu _ { 5 } \right) / 2 \\ & = \left( \left( \beta _ { 0 } + \beta _ { 1 } \right) + \left( \beta _ { 0 } + \beta _ { 2 } \right) + \left( \beta _ { 0 } + \beta _ { 3 } \right) \right) / 3 - \left( \left( \beta _ { 0 } + \beta _ { 4 } \right) + \left( \beta _ { 0 } + \beta _ { 5 } \right) \right) / 2 \\ & = \left( \beta _ { 1 } + \beta _ { 2 } + \beta _ { 3 } \right) / 3 - \left( \beta _ { 4 } + \beta _ { 5 } \right) / 2 \\ & = ( + 1 / 3 ) \cdot \beta _ { 1 } + ( + 1 / 3 ) \cdot \beta _ { 2 } + ( + 1 / 3 ) \cdot \beta _ { 3 } + ( - 1 / 2 ) \cdot \beta _ { 4 } + ( - 1 / 2 ) \cdot \beta _ { 5 } \end{aligned}
$$
- 群間の差は、重み付けされた群偏向の和。
- 群偏向の係数は足すとゼロ（上だと、$1/3,1/3,1/3,-1/2,-1/2$）。このような組を対比と呼ぶ。（正の係数、負の係数をそれぞれ足すと$＋１，ー１$になる）
- ここで求めた差をチェインの各ステップで $\sigma_y$で割ると効果量として表現できる。(よくわからない)

#### 結果の評価

<img src="./img/fig19_3_2.png"　width="500">

- plotMCMCの出力のいくつかが図19.3の下部（対比の事後分布）
- ２段あるうちの上段は差の事後分布、下段は効果量の事後分布
- 差の事後分布は寿命（日数）のスケールで表現されている。
- ROPE：実践的に等価な範囲(p342 12章)
    - 特定の状況において、パラメータの値が空値(帰無仮説が主張する値)と実質的に等価であるとみなされる小さな範囲のこと
    - ここでは差は０日、ROPEの範囲(差が0日と見れるような範囲)は−1.5日〜1.5日としている。
    - ROPE全体が、パラメータの事後分布の95%HDIの外側にある場合、そのパラメータの帰無仮説は棄却される
    - ROPE全体に、95％HDIが完全に内包されていた場合、パラメータの帰無仮説は実用的な目的に合ったものとして受け入れられる
- 統制群の平均と性に活発な群（一番右端の列）について見る。
    - 平均値の差は、最頻値で１５日と大きい
    - ROPEとは重なっていない。そのため帰無仮説０日は棄却される。
    - 効果量は1.0（これが大きいのかどうかよくわからない…）
- 統制群と処女1群(左から2列目)について見る。
    - 平均値の差の中央値は約７日。
    - 効果量は0.5をわずかに下回る。
    -  ROPEと重なっている。
     

#### 伝統的な分散分析
- いわゆる多重対比と呼ばれる検定が多かった
    - これは、「すべての群が同時に正確に等しい」ということが本当かどうかを問うもの
    - しかし、「すべての群が等しい」という帰無仮説が棄却された場合、次に気になるのはそれぞれの群の比較。
    - 「すべての群が等しい」という帰無仮説が棄却されなくても、一部の群間には有意な差があるかもしれない。そうなると個別の群について検定する必要がある。
- ここまで用いた階層ベイズ推定はすべての群の、有意義な比較検定をするというもの。伝統的な分散分析の多重対比とは異なる。

### 19.3.4. 多重比較と縮小
- さきほど、「気になる群間の比較はすべき」と述べたが、これは伝統的な帰無仮説の有意性検定の文脈における注意事項とは相反する。
    - 群間に差がなくても、何度も検定していればどこかで帰無仮説を棄却してしまう。(p331)
    - そのようなことを避けるために、フォールスアラーム率を5%ではなくより厳しくするか、もしくは比較を最小限に抑えるべき。
    - ベイジアン分析ではこの問題を避けられる。
- また、ベイジアン分析はモデルに事前の知識を組み込むことによってフォールスアラーム率を緩和することができる。

### 19.3.5. 2群の場合
- 現在のシナリオの中で特殊なケースは、群が２つしかない場合。
- 本セクションのモデルは原則として、２群の場合に適用できるが、あまり良くない。
    - 群の数が少ない場合、実質的に縮小が起こらないから。（わからない）
- 2群においては、セクション１６．３のモデルを用いることがより適切である。
<img src="./img/fig16_11.png">

- このようなモデルはデータの外れ値を調整するため、個別のt分布を用いている。そのため群間の異なる分散が許容される。
    - なお、階層的多群モデルでも、群間で異なる分散を表現したり、外れ値を調整するために、セクション19.5で一般化する。

## 19.4. 量的予測変数を含める

### 補助的な予測変数（共変量）を追加する。
<img src="./img/fig19_3_1.png" width="500">

- 図19.3では、各群のデータが大きな標準偏差を持っていた。
- もし群内分散が、別の測定可能な何かによる影響だったら、それを特徴量として取り入れることは、群間の差の検出性を改善するのに有用。
- 今回、取り入れる特徴：頑健性(胸部の大きさ)
    - 被験体の頑健性を測定し、それが寿命にどう影響を与えるかを見る。
    - このような量的予測変数を「共変量」と呼ぶ。
- 共変量
    - 概して、名義的予測変数の影響を分離させる、補助的な予測変数。
    - しかし数学的には、これらの名義的予測変数と量的予測変数はモデル上同等な立場。
- これを取り込むと式は下記のようになる
$$
\mu ( i ) = \beta _ { 0 } + \sum _ { j } \beta _ { [ j ] } x _ { [ j ] } ( i ) + \beta _ { \mathrm { cov } } x _ { \mathrm { cov } } ( i )
$$


### 共変量を入れる前、入れる後でダイアグラムはどう変わるか

- 図１９．２(共変量なし)
<img src="./img/fig19_2.png" width="500">

- 図19.4(共変量あり)
<img src="./img/fig19_4.png" width="500">

- 式が変わった。共変量の事前分布が追加された。

### 式をどのように捉えるべきか

- 式再掲
$$
\mu ( i ) = \beta _ { 0 } + \sum _ { j } \beta _ { [ j ] } x _ { [ j ] } ( i ) + \beta _ { \mathrm { cov } } x _ { \mathrm { cov } } ( i )
$$


- 上記の式では、ベースライン$\beta_0$は２つの見方を期待されている。（が、まだそう見てはいけない）
    - 予測されたデータ全体の平均（名義的偏向はゼロ和）
    - $x_{\rm cov}$の回帰曲線の切片
    - ただ上記のままだと、データ全体の平均にはなっていない。そこで書き直す。
    
$$
\begin{aligned} \mu & = \alpha _ { 0 } + \sum _ { j } \alpha _ { [ j ] } x _ { [ j ] } + \alpha _ { \mathrm { cov } } \left( x _ { \mathrm { cov } } - \overline { x } _ { \mathrm { cov } } \right) \\ & = \underbrace { \alpha _ { 0 } + \overline { \alpha } - \alpha _ { \mathrm { cov } } \overline { x } _ { \mathrm { cov } } } _ { \beta _ { 0 } } + \sum _ { j } \underbrace { \left( \alpha _ { [ j ] } - \overline { \alpha } \right) } _ { \beta _ { [ j ] } } x _ { [ j ] } + \underbrace { \alpha _ { \mathrm { cov } } } _ { \beta _ { \mathrm { cov } } } x _ { \mathrm { cov } } \end{aligned}
$$

$$
\overline { \alpha } = \frac { 1 } { J } \sum _ { j = 1 } ^ { J } \alpha _ { [ j ] }
$$

### 19.4.1. 例：性別、死亡率、身体の大きさ
- ショウジョウバエの例に、共変量を追加することを考える。
- 図19.3では、群内分散が大きく、それをもとに小さな群間の差を推定しようと若干無茶している。
<img src="./img/fig19_3_1.png" width="500">
- また、交尾の機会がないオスの群と処女１群を比べると、
    - 7日の寿命の違いが示唆される一方
    - 推定の不確実性は大きい（差の９５％HDI区間は0~14日程度）
    - これを改善したい
<img src="./img/fig19_3_2.png" width="500">
- 寿命についての群内分散は大きさからくる変動が大きい。そこでショウジョウバエの胸部の大きさを共変量として用いる

#### 分析結果
<img src="./img/fig19_5.png" width="500">

- 上段の図は各群での、胸部の大きさと寿命でプロットしたもの。
    - 重ねられた線は、MCMCチェインの様々なステップにおけるパラメータをもとに、$\beta_0 + \beta_{[j]} + \beta_{\rm cov} x_{\rm cov}$ を描いたもの
    - 描かれている正規分布は各線における$\sigma_y$の値に対応している
- 着目すべき点
    - 群内ノイズの標準偏差が、共変量のない前回の分析と比較して小さいこと
        - $\sigma_y$の最頻値は約10.5日
    - 交尾の機会がないオスの群と処女１群との間の差の、９５％HDI区間は４日〜１４日。
        - 差異０の場合が排除されており、共変量がない場合よりも確かな推定

### 19.4.2. 従来のANCOVAとの類似点
- 従来の頻度主義的な手法では、上記のようなモデル記述はANCOVAと呼ばれていた。
- ベイズ的手法はANCOVAに似ているが、そのものではない。
- 頻度主義の技術者は、
    - すべての群において傾きが等しい
    - すべての群において標準偏差が等しい
    - 誤差が正規分布に従っていない可能性がある
- といった仮定のために、p値を用いた検定を行うようせき立てられている
- ベイジアンアプローチでは、これらの問題に対処するため、記述的モデルが一般化される

### 19.4.3. 階層線形回帰との関係
- セクション17.3での階層線形回帰と類似している。
- 図17.5と図19.5は類似している
<img src="./img/fig17_5.png" width="500">



#### - セクション17.3のモデルと図19.5でのモデルに違いは、量的予測変数における傾きの係数
    - 17.3では、各個人に固有の傾きが割り振られていて、その固有の傾きはさらに高次元の分布で影響を与えあっていた
    - 図19.5のモデルはすべての群は量的予測変数（共変量）について同じ傾きを用いて表されていた
<img src="./img/fig17_6.png" width="500">

<img src="./img/fig19_4.png" width="500">

- 概念的には、モデル間の主要な差異は単に取り上げられる焦点の違いである。
    - 階層線形回帰モデルでは傾きの係数に焦点が当てられていた。
    - 本セクションで最も注目しているのは切片と群間の差異。それに付随して共変量の傾きに注目している。

## 19.5. 異なる分散と外れ値に対する頑健性
- 群内のデータは正規分布を想定しており、群間は等分散性を仮定していた
    - これは単なる単純化と、従来のANOVAとの一貫性のため。
    - ベイズ用ソフトウェアでは上記以外を用いることもできる。
- 本セクションでは、
    - 正規分布の代わりにt分布
    - 等分散ではなく、固有の標準偏差パラメータを各群に割り当てる
    - 階層的な事前分布を標準偏差パラメータに設定

<img src="./img/fig19_6.png" width="500">

- 図19.6は図19.2の単なる拡張(比較のため19.2を再掲)
<img src="./img/fig19_2.png" width="500">

- 図１９．６(再掲)
<img src="./img/fig19_6.png" width="500">

- t分布は正規化パラメータ$\nu$を持っている。その事前分布が追加されている
- t分布のスケールパラメータの階層的事前分布もある
    - すべての群に適用される１つの$\sigma_y$パラメータ(図19.2)の代わりに、各群は固有のスケールパラメータ$\sigma_j$を持っている。

### 19.5.1. 例：異なる分散における平均値の比較
#### データの概要と等分散を仮定したモデルの適用結果
- 各群が異なる分散を持ったモデルにすることで、どんなメリットが有るのか見ていく。
- データの説明
    - A,B,C,Dの４つの群
    - 平均値は97,99,102,104
    - 標準偏差はA,Dが大きく、B,Cが小さい
    - データは正規分布からランダムにサンプリングされたもの。（下記の赤丸）
- これまで同様、等分散であると仮定されたモデルを適用した結果は図19.7
<img src="./img/fig19_7_1.png" width="500">
- 群内の標準偏差の推定が、小さな分散の群と大きな分散の群の妥協の産物となってしまっている

- 差の事後分布をみると、直感と異なる結果となってしまっている。
<img src="./img/fig19_7_2.png" width="500">

- $\mu_D - \mu_A$の事後分布は、差異が明らかに０より大きい一方で、
- $\mu_C - \mu_B$の事後分布は、差異が正の値である場合、95%HDI区間内に０があるため、十分に不確定であることが示される。
- しかし、この結果は直感に反する。
    - B群とC群は重なりも少なく、差は0でないというべき
    - A群とD群は重なりも大きく、中心傾向はグラフと異なるように思われる

#### 直感と異なる結果が生じた理由
- 等分散を仮定していることに伴い、$\mu_C$と$\mu_B$に用いている標準偏差が本来より大きくなってしまっているから
- それによって、$\mu_C$と$\mu_B$の値が何度も上下してしまう。
- そのため、$\mu_C - \mu_B$の事後分布も広くなってしまう。
- $\mu_D$と$\mu_A$は逆に、標準偏差が本来より小さい。そのため、$\mu$の推定値がデータの中心からあまり動かなくなる。
- そのため、$\mu_D - \mu_A$の推定が強まってしまう。

#### 分散が異なることを仮定したモデルでの結果
- では、各群において分散の異なるモデルを用いてみよう。その結果が図19.8。
<img src="./img/fig19_8_1.png" width="500">

- 推定された標準偏差は、BとCのものが、AとDより小さくなっており、適切になっている。

- さらに、平均値の差の事後分布もまた適切になっている。
<img src="./img/fig19_8_2.png" width="500">

- $\mu_C - \mu_B$の差の事後推定は０より大きくなっている。
- $\mu_D - \mu_A$の差の事後推定は0付近のROPEとオーバーラップした95%HDIとなっている。
    - 2群のデータはかなりオーバーラップしていて、サンプルサイズも小さいため、理にかなっている
    
- また、各群は固有の推定スケール($\sigma_i$)を有しているため、群間でのスケールの差を調べることができる。